In [14]:
using ITensors, ITensorMPS
import Pkg
Pkg.activate(".")
using ITensors

function addOp(arr, op, site)
    push!(arr, op)
    push!(arr, site)
end

function Q_ij(i,j,n)

    temp = []
    if i==j
        coeff = 1/(2*sqrt(2*i*(i-1)))
        for c in 1:i-1
            addOp(temp, "Z", 3*(n-1)+c)
            addOp(temp, "Z", 3*(n-1)+i)
        end
    else
        si = 3*(n-1)+i
        sj = 3*(n-1)+j
        addOp(temp, "S+", si)
        for k in sj:si
            addOp(temp, "Z", k)
        end
        addOp(temp, "S+", sj)
    end
    pushfirst!(temp, coeff)
    return tuple(temp...)
end

function QinQim(n,m)
    H = OpSum()
    for i in 2:3
        a = Q_ij(i,i,n)
        b = Q_ij(i,i,m)
        coeff = a[1] * b[1]
        ops_sites = Any[]
        append!(ops_sites, collect(a[2:end]))
        append!(ops_sites, collect(b[2:end]))
        H += coeff, tuple(ops_sites...)...
    end
    return H
end



# Main block
let
    m = 1
    N = 24
    sites = siteinds("S=1/2", N)
    states = [isodd(n) ? "Up" : "Dn" for n in 1:N]
    psi0 = MPS(sites, states)

    # --- Mass Term ---
    H_mass = OpSum()
    for j in 2:(N - 1)
        coeff = (m / 2) * (isodd(div(j,3)) ? 1 : -1)
        H_mass += coeff, "Z", j
    end

    # --- Electric Field Term ---
    H_elec = OpSum()
    for n in 1:(N - 1)
        Q_ij_term = QinQim(n, n )
    end

    # --- Combine all Hamiltonian terms symbolically ---
    H_total = H_mass + H_elec

    # Convert once to MPO
    H = MPO(H_total, sites)

    # --- Run DMRG ---
    nsweeps = 5
    maxdim = [10, 20, 100, 100, 200]
    cutoff = 1.0e-10

    energy, psi = dmrg(H, psi0; nsweeps, maxdim, cutoff)
    println("Final energy = $energy")
end


  Activating project at `c:\Users\aradh\Desktop\Physic 697\physics_697_fall_2025_Max_Aradh_project`


After sweep 1 energy=4.0  maxlinkdim=1 maxerr=0.00E+00 time=24.504
After sweep 2 energy=4.0  maxlinkdim=1 maxerr=0.00E+00 time=0.079
After sweep 3 energy=4.0  maxlinkdim=1 maxerr=0.00E+00 time=0.041
After sweep 4 energy=4.0  maxlinkdim=1 maxerr=0.00E+00 time=0.031
After sweep 5 energy=4.0  maxlinkdim=1 maxerr=0.00E+00 time=0.035
Final energy = 4.0
